# MQTT Signal im lokalen Netz empfangen
- ACHTUNG: die Host-IP adresse weicht von der in der Anleitung ab, sobald man nicht direkt den Laptop mit dem CMTK verbindet sondern über den Router das CMTK ins Netzwerk einbindet
- Sobald über die Konfiguration (http://cmtk/#/device-settings/0/database) das Datenerfassungsintervall verändert wird, ist das an den Zeitabständen des empfangenen MQTT Signals erkennbar
- Die Anleitung im Developer Guide ist sehr hilfreich! https://dev.cmtk.de/1.6.0/getting-started/python/

## Zugang ohne Benutzername und Passwort aktiv

Dieser Code funktioniert nur wenn in der MQTT-Konfiguration die Option "Zugang ohne Benutzername und Passwort aktiv" aktiviert ist

In [ ]:
import paho.mqtt.client as mqtt

if __name__ == '__main__':
    # Parameters
    host = "192.168.0.7"
    port = 1883

    # Message callback
    def on_message(client, userdata, msg):
        print("Topic: " + msg.topic + "\nPayload: " + str(msg.payload) + "\n")

    # Connection callback
    def on_connect(client, userdata, flags, rc):
      client.subscribe("cmtk/cmtkv1/port1/pd")  

    try:
      # Connect 
      client = mqtt.Client()
      client.on_connect = on_connect
      client.on_message = on_message
      client.connect(host, port, 6)

      # Loop
      client.loop_forever()

      # Disconnect
      client.disconnect()
    except:
        print("MQTT error.")


## MQTT ohne TLS aktiv

- Dieser Code funktioniert nur wenn in der MQTT-Konfiguration die Option "MQTT ohne TLS aktiv" aktiviert ist
- Das Passwort muss in der Konfiguration gesetzt werden

In [ ]:
client.username_pw_set("user", "test123")

client.connect(host, port, 6)
# Loop
client.loop_forever()
# Disconnect
client.disconnect()

## MQTT mit TLS aktiv

- Dieser Code funktioniert nur wenn in der MQTT-Konfiguration die Option "MQTT mit TLS aktiv" aktiviert ist
- das Zertifikat kann über die Konfiguration heruntergeladen werden


In [ ]:
!pwd

- das funktioniert nur wenn ich für broker_adress nicht die ip adresse (192.168.0.7) eingebe sondern den aufgelösten hostname "cmtk". Den aufgelösten hostname erhalte ich z.B. in der shell mit "host 192.168.0.7"

In [ ]:
import ssl
import paho.mqtt.client as mqtt

# MQTT Broker information

broker_address = "cmtk"  # replace with your broker address
port = 8883  # specify the TLS port
topic = "cmtk/cmtkv1/port1/pd"  # replace with the desired topic

# TLS settings
ca_cert = "/Users/esentri/Documents/07_InterneProjekte/07_Balluff_Sensorik/ca.crt"  # replace with the path to your CA certificate

# Callback when the client receives a CONNACK response from the broker
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe(topic)

# Callback when a message is received from the broker
def on_message(client, userdata, msg):
    print(f"Received message on topic {msg.topic}: {msg.payload.decode()}")

# Create a MQTT client instance
client = mqtt.Client()

# Set TLS options with only the CA certificate
client.tls_set(ca_certs=ca_cert, cert_reqs=ssl.CERT_REQUIRED)

# Set callbacks
client.on_connect = on_connect
client.on_message = on_message

# Connect to the broker
client.connect(broker_address, port, keepalive=200)

# Loop to maintain the connection and process messages
client.loop_forever()
